### Imports

In [ ]:
import cv2
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from PIL.ImageOps import invert
from own_utils import remove_overlapping_junctions, non_max_suppression_fast

In [ ]:
show_intermediate = False   # set to True to show intermediate results

### Image preprocessing

In [ ]:
test_image = '../assets/example_image2.jpg'

# convert to grayscale
img = Image.open(test_image).convert('L')
img = invert(img)
img = ImageEnhance.Contrast(img).enhance(2)
img = img.point(lambda p: p > 220 and 255)
img = img.filter(ImageFilter.SMOOTH)

# show image
if show_intermediate:
    img.show()

### Components Inference

In [ ]:
# Get the model
model_path = '../models/components.pt' 
c_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
c_model.eval()

In [ ]:
import pandas as pd
import numpy as np

def non_max_suppression_fast(boxes, scores, iou_threshold=0.5):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    scores = np.array(scores)

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    indices = np.argsort(scores)[::-1]

    keep = []

    while len(indices) > 0:
        current = indices[0]
        keep.append(current)

        xx1 = np.maximum(x1[current], x1[indices[1:]])
        yy1 = np.maximum(y1[current], y1[indices[1:]])
        xx2 = np.minimum(x2[current], x2[indices[1:]])
        yy2 = np.minimum(y2[current], y2[indices[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[indices[1:]]

        suppressed_indices = np.where(overlap <= iou_threshold)[0]

        indices = indices[suppressed_indices + 1]

    return keep

# Inference
c_results = c_model(img)

# Print and show results
if show_intermediate:
    print(c_results.pandas().xyxy)
    c_results.show()

### Junction inference

In [ ]:
model_path = '../models/junctions.pt'
j_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
j_model.eval()

In [ ]:
# Perform inference on the image without components
j_results = j_model(img)

# Print and show results
if show_intermediate:
    print(j_results.pandas().xyxy)
    j_results.show()

### Post processing

In [ ]:
# Remove overlapping junctions
c_coords, j_coords = remove_overlapping_junctions(j_results, c_results, overlap_threshold=0.1)

# Remember the old junctions
columns = ['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']
old_j_df = pd.DataFrame(j_coords, columns=columns)
old_c_df = pd.DataFrame(c_coords, columns=columns)

# Perform non-maximum suppression on coords and junctions
c_coords = non_max_suppression_fast(c_coords, iou_threshold=0.5)
j_coords = non_max_suppression_fast(j_coords, iou_threshold=0.5)

# Create dataframes from the components and remaining junctions
c_df = pd.DataFrame(c_coords, columns=columns)
j_df = pd.DataFrame(j_coords, columns=columns)

if show_intermediate:
    print("Old components:")
    print(old_c_df)
    
    print("Components:")
    print(c_df)

    print("Old junctions:")
    print(old_j_df)

    print("NMS Junctions:")
    print(j_df)

### Show final detections

In [ ]:
# using c_df containing the components and j_df containing the junctions, draw all bounding boxes on the original image
img = cv2.imread(test_image)
c_labels = c_model.model.names
j_labels = j_model.model.names

# Draw components
for index, row in c_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    label = c_labels[int(row['class'])]
    cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Draw junctions
for index, row in j_df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

    label = j_labels[int(row['class'])]
    cv2.putText(img, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Save the image
cv2.imwrite('../assets/output.jpg', img)

### Convert to generated image
Take the final output list and generate the digital circuit based on that

In [ ]:
# TODO